In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from processingQst import *
from similarityFcts import *
from tqdm import tqdm
import pandas as pd

In [ ]:
df = pd.read_csv("./input/csv/train.csv",delimiter=";")

Qst = df["question"]
id  = df["id"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]
nbrRep = 5
print(len(Qst))
data_list = []
RepListIndx = 'abcde'

In [ ]:
def writeJson(path,data):
    with open(path,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4,ensure_ascii=False)

In [ ]:
# keyword = ['bactérie','méningite']
with open('output/corpusMS.json', 'r') as f:
    corpusA = json.load(f)
with open('output/corpusMerck.json', 'r') as fs:
    corpusB = json.load(fs)

In [ ]:

# for cpt in range(len(Qst)):
for cpt in tqdm(range(len(Qst))):
    if (cpt > 300):
        break
    elif (cpt>=0): 
        qst = Qst[cpt]
        # print(qst)
        nwQst = ''
        listMed = []
        keywordQ = []
        keywordQP = []
        ListMedAns = ''
        Spword = False
        data_dict = {}

        # Détécter la négation 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
            # print(WrongQst)
        else: 
            WrongQst = False 
            # print(WrongQst)
        
        # supprimer les parenthèses 
        qst = del_betParenthese(qst)

        # fragementation des questions selon les ","
        fragQst = splitQst(qst)

        for icpt in range(len(fragQst)):
            if not (fragQst[icpt]==''):
                nwQst,Spword,listMed = recoverMedFrag(fragQst,icpt,nwQst,Spword)
                keywordQ.extend(listMed)
        nwQst = nwQst.lstrip()
        nwQstp = ''
        var = ''
        for j in range(5):
            keywordQP = []
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            # Ans = re.sub(r'([^\d\W]+)(\W*)(\d*)', Ans)
            # Ans = re.sub(r'([^\d\W]+)(\W*)(\d*)', r'\1 \2\3 ', Ans)
            # Ans = re.sub(r"\s+", " ", Ans)
            # print(Ans)
            keywordQP.extend(RecoverListMed(Ans))
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            # Appliquer la négation 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            
            # # test de similarité 
            # print("****")
            # print(nwQstp)
            # print(keywordQ)
            # print(keywordQP)
            # print("====")
            taux_sim = getRepDB(keywordQ,keywordQP,corpusA,nwQstp)
            taux_simB = 0# getRepDB(keywordQ,keywordQP,corpusB,nwQstp)
            if taux_sim>0.9:# or taux_simB>0.81:
                if var !="":
                    var = var + '|'
                var = var + RepListIndx[j]
            
        # créer un dataframe pour les données actuelles
        data_dict = {'id': id[cpt], 'correct_answers': var}
        data_list.append(data_dict)



In [ ]:
# writeJson("output/Qst_dev.json",data_list)
# ---
df = pd.DataFrame(data_list)
df.to_csv('./output/tachePrincipale.csv', sep=';', index=False)